In [ ]:
2from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
EP6 = pd.read_csv("/content/drive/MyDrive/ Thesis 2023/EP6_total2.csv")
EP7 = pd.read_csv("/content/drive/MyDrive/ Thesis 2023/EP7_total2.csv")
EP8 = pd.read_csv("/content/drive/MyDrive/ Thesis 2023/EP8_total2.csv")


In [ ]:
EP6_2 = pd.read_csv("/content/drive/MyDrive/ Thesis 2023/EP6_RCVs.csv")
EP7_2 = pd.read_csv("/content/drive/MyDrive/ Thesis 2023/EP7_RCVs.csv")
EP8_2 = pd.read_csv("/content/drive/MyDrive/ Thesis 2023/EP8_RCVs.csv")

In [ ]:
EP6_T = pd.read_csv("/content/drive/MyDrive/ Thesis 2023/EP6_T2.csv")

In [ ]:
# Fill NaN values with empty strings
EP6.fillna('', inplace=True)

In [ ]:
import numpy as np


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

from nltk import word_tokenize, WordNetLemmatizer#, pos_tag, pos_tag_sents, sent_tokenize
from nltk.corpus import stopwords#, wordnet

class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
      res=[]
      for token in word_tokenize(doc):
        word=''.join(filter(str.isalnum, token))
        if ((word not in self.ignore_tokens) and (len(self.wnl.lemmatize(word))>2) and not (any(char.isdigit() for char in word))): 
          res.append(self.wnl.lemmatize(word))
      return res

stop_words = stopwords.words('english')
#stop_words.extend(["more","stop","words"])

tokenizer = LemmaTokenizer()
token_stop = list(set(tokenizer(' '.join(stop_words))))
token_stop.sort()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
EP6.isnull().count()


Unnamed: 0          1253
Vote ID             1253
raw_text            1253
type_of_vote        1253
main_policy_name    1253
                    ... 
95280               1253
95281               1253
95704               1253
96101               1253
23852               1253
Length: 947, dtype: int64

## Preprocessing of the data EP6

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
# define the columns to transform
text_columns = ['raw_text2', 'type_of_vote', 'main_policy_name', 'committee_code']

# define the transformer
transformer = ColumnTransformer(
    transformers=[
        ('raw_text2', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'raw_text2'),
        ('type_of_vote', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'type_of_vote'),
        ('main_policy_name', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'main_policy_name'),
        ('committee_code', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'committee_code')
    ])

# fit and transform the data
tfidf_array = transformer.fit_transform(EP6[text_columns])




/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
from sklearn.decomposition import TruncatedSVD
number_dimensions = 45
lsa_model = TruncatedSVD(n_components=number_dimensions, random_state=420)
lsa_comps = lsa_model.fit_transform(tfidf_array)
print(lsa_model.explained_variance_ratio_)
print(lsa_model.explained_variance_ratio_.sum())

[0.08878754 0.12248197 0.07653322 0.06793096 0.0641246  0.05728278
 0.05354004 0.0495635  0.03293544 0.02861935 0.02655948 0.02349873
 0.02154625 0.02001938 0.01713696 0.01587952 0.0084174  0.00790274
 0.0071001  0.00643547 0.00570467 0.00511605 0.00477595 0.00462822
 0.00420155 0.00401022 0.00392292 0.00370708 0.00356567 0.00331549
 0.00322059 0.00311401 0.00296691 0.00285861 0.00276918 0.00258087
 0.00246918 0.00241332 0.00240079 0.00232857 0.00224494 0.00219623
 0.00208964 0.002024   0.00196762]
0.8768877369731358


## Create a model for each MEP--> EP6
Changes:
> max_features = 9000 the avg accuracy is 69% with logistic regression and 40 on SVD

> max_features = 9000 the avg accuracy is 71% with logistic regression and 40 on SVD


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

# Create a dictionary to store MEP names as keys and accuracy values as values
dummy_results = {}

# Loop through each MEP column and train a separate model for each MEP
for col in EP6.columns[7:]:
    # Step 1: Prepare the data
    X = lsa_comps # Input features (TF-IDF + truncated SVD features)
    y = EP6[col].values # Target variable (voting behavior of the MEP)
    
    # Check the number of unique classes in the target variable
    num_classes = len(np.unique(y))
    if num_classes < 2:
        print(f"Skipping MEP {col} due to only one class present.")
        continue
    
    # Step 2: Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Step 3: Choose and configure the model
    model = Pipeline([
        ('scaler', StandardScaler()), # You can apply additional preprocessing steps as needed
        ('classifier', DummyClassifier(strategy="most_frequent")) # Change the classifier as needed (e.g., DecisionTreeClassifier, RandomForestClassifier, SVC, MLPClassifier)
    ])
    
    # Step 4: Train the model
    model.fit(X_train, y_train)
    
    # Step 5: Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    #print(f"Accuracy for MEP {col}: {accuracy}")
    
    # Store accuracy result and feature importances in the accuracy_results dictionary
    dummy_results[col] = {'accuracy': accuracy}

# Convert the accuracy_results dictionary to a DataFrame
dummy_df = pd.DataFrame.from_dict(dummy_results, orient='index')

# Add the MEP column to the DataFrame
dummy_df = dummy_df.reset_index().rename(columns={'index': 'MEP'})




Skipping MEP 72754 due to only one class present.


In [ ]:
dummy_df.describe()

,accuracy
count,939.000000
mean,0.661630
std,0.160836
min,0.231076
25%,0.545817
50%,0.673307
75%,0.790837
max,1.000000


In [ ]:
# Get the minimum and maximum accuracy values
min_accuracy = dummy_df['accuracy'].min()
max_accuracy = dummy_df['accuracy'].max()
median_accuracy = dummy_df['accuracy'].median()
mean_accuracy = dummy_df['accuracy'].mean()

# Print the results
print(f"Minimum Accuracy: {min_accuracy}")
print(f"Maximum Accuracy: {max_accuracy}")
print(f"Median Accuracy: {median_accuracy}")
print(f"Mean Accuracy: {mean_accuracy}")


Minimum Accuracy: 0.23107569721115537
Maximum Accuracy: 1.0
Median Accuracy: 0.6733067729083665
Mean Accuracy: 0.661630368833505


## Preprocessing of the data EP7




In [ ]:
# Fill NaN values with empty strings
EP7.fillna('', inplace=True)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
# define the columns to transform
text_columns = ['raw_text2', 'Type of Vote', 'Committee responsabile', 'De']

# define the transformer
transformer = ColumnTransformer(
    transformers=[
        ('raw_text2', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'raw_text2'),
        ('Type of Vote', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'Type of Vote'),
        ('Committee responsabile', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'Committee responsabile'),
        ('De', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'De')
    ])

# fit and transform the data
tfidf_array = transformer.fit_transform(EP7[text_columns])




/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
from sklearn.decomposition import TruncatedSVD
number_dimensions = 45
lsa_model = TruncatedSVD(n_components=number_dimensions, random_state=420)
lsa_comps = lsa_model.fit_transform(tfidf_array)
print(lsa_model.explained_variance_ratio_)
print(lsa_model.explained_variance_ratio_.sum())

[0.02452856 0.17430414 0.10193883 0.06797607 0.05618554 0.04990522
 0.04688647 0.04049931 0.03979942 0.03774107 0.02449297 0.02161396
 0.01783039 0.01453779 0.01009708 0.00946989 0.00731835 0.00673091
 0.00658367 0.00534637 0.00515777 0.0042157  0.00411214 0.00381659
 0.00338214 0.00336178 0.00311808 0.00294587 0.0028723  0.00273721
 0.00263551 0.00254997 0.00244213 0.00239303 0.00226969 0.00217931
 0.00210151 0.00206324 0.00197609 0.00191507 0.00187631 0.00179047
 0.00171927 0.001665   0.00160649]
0.8306886666479715


## Create a model for each MEP--> EP7
Changes:
> max_features = 9000 the avg accuracy is 69% with logistic regression and 40 on SVD

> max_features = 9000 the avg accuracy is 71% with logistic regression and 40 on SVD


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

# Create a dictionary to store MEP names as keys and accuracy values as values
dummy_results = {}

# Loop through each MEP column and train a separate model for each MEP
for col in EP7.columns[7:]:
    # Step 1: Prepare the data
    X = lsa_comps # Input features (TF-IDF + truncated SVD features)
    y = EP7[col].values # Target variable (voting behavior of the MEP)
    
    # Check the number of unique classes in the target variable
    num_classes = len(np.unique(y))
    if num_classes < 2:
        print(f"Skipping MEP {col} due to only one class present.")
        continue
    
    # Step 2: Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Step 3: Choose and configure the model
    model = Pipeline([
        ('scaler', StandardScaler()), # You can apply additional preprocessing steps as needed
        ('classifier', DummyClassifier(strategy="most_frequent")) # Change the classifier as needed (e.g., DecisionTreeClassifier, RandomForestClassifier, SVC, MLPClassifier)
    ])
    
    # Step 4: Train the model
    model.fit(X_train, y_train)
    
    # Step 5: Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    #print(f"Accuracy for MEP {col}: {accuracy}")
    
    # Store accuracy result and feature importances in the accuracy_results dictionary
    dummy_results[col] = {'accuracy': accuracy}

# Convert the accuracy_results dictionary to a DataFrame
dummy_df = pd.DataFrame.from_dict(dummy_results, orient='index')

# Add the MEP column to the DataFrame
dummy_df = dummy_df.reset_index().rename(columns={'index': 'MEP'})




Skipping MEP 736 due to only one class present.
Skipping MEP 737 due to only one class present.


In [ ]:
dummy_df.describe()

,accuracy
count,851.000000
mean,0.675383
std,0.158638
min,0.246246
25%,0.543544
50%,0.717718
75%,0.801802
max,1.000000


In [ ]:
# Get the minimum and maximum accuracy values
min_accuracy = dummy_df['accuracy'].min()
max_accuracy = dummy_df['accuracy'].max()
median_accuracy = dummy_df['accuracy'].median()
mean_accuracy = dummy_df['accuracy'].mean()

# Print the results
print(f"Minimum Accuracy: {min_accuracy}")
print(f"Maximum Accuracy: {max_accuracy}")
print(f"Median Accuracy: {median_accuracy}")
print(f"Mean Accuracy: {mean_accuracy}")


Minimum Accuracy: 0.24624624624624625
Maximum Accuracy: 1.0
Median Accuracy: 0.7177177177177178
Mean Accuracy: 0.6753827858410703


## Preprocessing of the data EP8




In [ ]:
# Fill NaN values with empty strings
EP8.fillna('', inplace=True)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
# define the columns to transform
text_columns = ['raw_text2', 'Type of Vote', 'Committee responsabile', 'De/Policy area']

# define the transformer
transformer = ColumnTransformer(
    transformers=[
        ('raw_text2', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'raw_text2'),
        ('Type of Vote', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'Type of Vote'),
        ('Committee responsabile', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'Committee responsabile'),
        ('De/Policy area', TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer, lowercase=True, strip_accents='unicode', min_df=0.05, max_df=0.9, max_features=9000, ngram_range=(1,3)), 'De/Policy area')
    ])

# fit and transform the data
tfidf_array = transformer.fit_transform(EP8[text_columns])




/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
from sklearn.decomposition import TruncatedSVD
number_dimensions = 45
lsa_model = TruncatedSVD(n_components=number_dimensions, random_state=420)
lsa_comps = lsa_model.fit_transform(tfidf_array)
print(lsa_model.explained_variance_ratio_)
print(lsa_model.explained_variance_ratio_.sum())

[0.01678534 0.17400072 0.13209778 0.06625583 0.05989513 0.04797599
 0.04660548 0.04347753 0.03661654 0.0289421  0.02146067 0.01992943
 0.01472099 0.01111243 0.00988215 0.00844216 0.00740817 0.00638199
 0.00617594 0.0057706  0.00535407 0.00512328 0.00495514 0.00475831
 0.0043276  0.00412326 0.0038976  0.00346097 0.00342858 0.00314649
 0.00304245 0.00301303 0.00280876 0.00265719 0.00263341 0.00250744
 0.0023631  0.00227674 0.00225423 0.00212505 0.00200921 0.00195892
 0.00192156 0.00183555 0.00177745]
0.8416963515685937


## Create a model for each MEP--> EP7
Changes:
> max_features = 9000 the avg accuracy is 69% with logistic regression and 40 on SVD

> max_features = 9000 the avg accuracy is 71% with logistic regression and 40 on SVD


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

# Create a dictionary to store MEP names as keys and accuracy values as values
dummy_results = {}

# Loop through each MEP column and train a separate model for each MEP
for col in EP8.columns[7:]:
    # Step 1: Prepare the data
    X = lsa_comps # Input features (TF-IDF + truncated SVD features)
    y = EP8[col].values # Target variable (voting behavior of the MEP)
    
    # Check the number of unique classes in the target variable
    num_classes = len(np.unique(y))
    if num_classes < 2:
        print(f"Skipping MEP {col} due to only one class present.")
        continue
    
    # Step 2: Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Step 3: Choose and configure the model
    model = Pipeline([
        ('scaler', StandardScaler()), # You can apply additional preprocessing steps as needed
        ('classifier', DummyClassifier(strategy="most_frequent")) # Change the classifier as needed (e.g., DecisionTreeClassifier, RandomForestClassifier, SVC, MLPClassifier)
    ])
    
    # Step 4: Train the model
    model.fit(X_train, y_train)
    
    # Step 5: Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    #print(f"Accuracy for MEP {col}: {accuracy}")
    
    # Store accuracy result and feature importances in the accuracy_results dictionary
    dummy_results[col] = {'accuracy': accuracy}

# Convert the accuracy_results dictionary to a DataFrame
dummy_df = pd.DataFrame.from_dict(dummy_results, orient='index')

# Add the MEP column to the DataFrame
dummy_df = dummy_df.reset_index().rename(columns={'index': 'MEP'})




Skipping MEP 124795 due to only one class present.
Skipping MEP 28407 due to only one class present.
Skipping MEP 193706 due to only one class present.


In [ ]:
dummy_df.describe()

,accuracy
count,855.000000
mean,0.680518
std,0.165102
min,0.258454
25%,0.543478
50%,0.724638
75%,0.818841
max,1.000000


In [ ]:
# Get the minimum and maximum accuracy values
min_accuracy = dummy_df['accuracy'].min()
max_accuracy = dummy_df['accuracy'].max()
median_accuracy = dummy_df['accuracy'].median()
mean_accuracy = dummy_df['accuracy'].mean()

# Print the results
print(f"Minimum Accuracy: {min_accuracy}")
print(f"Maximum Accuracy: {max_accuracy}")
print(f"Median Accuracy: {median_accuracy}")
print(f"Mean Accuracy: {mean_accuracy}")


Minimum Accuracy: 0.2584541062801932
Maximum Accuracy: 1.0
Median Accuracy: 0.7246376811594203
Mean Accuracy: 0.6805181230047743
